In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline

In [ ]:
parYld_df = pd.read_csv('../data/yield-curve-rates-1990-2024.csv')

# drop short rates
parYld_df = parYld_df.drop(columns=['1 Mo', '2 Mo', '3 Mo', '4 Mo'], axis=1)
parYld_df['Date'] = pd.to_datetime(parYld_df['Date'], format='%m/%d/%y')
parYld_df.iloc[:, 1:] = parYld_df.iloc[:, 1:] / 100 # convert percentage to decimal

# remove any rows with missing rates
parYld_df = parYld_df.dropna(axis=0, how='any', subset=parYld_df.columns[1:])

In [5]:
parYld_T = parYld_df.set_index('Date').T

# Set first index to 0.5, convert others by removing 'yr' and casting to float
new_index = [0.5 if idx == '6 Mo' else float(idx.replace('Yr', '')) for idx in parYld_T.index]
parYld_T.index = new_index

In [6]:
# cubic spline interpolation
new_index = np.arange(0.5, 30.5, 0.5) # quarterly intervals from 6 months to 30 years

parYld_interp = pd.DataFrame(index=new_index, columns=parYld_T.columns)

for col in parYld_T.columns:
    x = parYld_T.index
    y = parYld_T[col].values
    mask = ~np.isnan(y) # drop na
    if not np.all(mask):
        print(f"NaNs found in column '{col}'; interpolating over available data.")
    cs = CubicSpline(x[mask], y[mask], bc_type='natural')
    parYld_interp[col] = cs(new_index)

In [23]:
def par_to_spot(par_rates, maturities):
    """
    Convert par rates to spot rates using bootstrapping.
    
    Parameters:
    par_rates (array-like): Par rates for different maturities.
    maturities (array-like): Corresponding maturities in years.
    
    Returns:
    np.ndarray: Spot rates corresponding to the maturities.
    """
    n = len(par_rates)
    spot_rates = np.zeros(n)
    last_disc = 0.0
    sum_zcb = 0.0

    for i in range(n):
        if i == 0:
            spot_rates[i] = par_rates[i]
        else:
            last_disc = 1 / (1 + spot_rates[i-1])**maturities[i-1]
            sum_zcb += last_disc
            accum_n = (1 + par_rates[i] / 2) / (1 - par_rates[i] / 2 * sum_zcb)
            spot_rates[i] = accum_n**(1/maturities[i]) - 1
    return spot_rates

In [25]:
def spot_to_fwd(spot_rates, maturities):
    """
    Convert spot rates to forward rates.
    
    Parameters:
    spot_rates (array-like): Spot rates for different maturities.
    maturities (array-like): Corresponding maturities in years.
    
    Returns:
    np.ndarray: Forward rates corresponding to the maturities.
    """
    n = len(spot_rates)
    fwd_rates = np.zeros(n)
    accum_n = 0.0
    accum_lastn = 0.0

    for i in range(n):
        if i == 0:
            fwd_rates[i] = spot_rates[i]
        else:
            accum_n = (1 + spot_rates[i])**maturities[i]
            accum_lastn = (1 + spot_rates[i-1])**maturities[i-1]
            fwd_rates[i] = (accum_n / accum_lastn)**(1/(maturities[i]-maturities[i-1])) - 1
    return fwd_rates

In [ ]:
spot_array = np.empty(parYld_interp.shape) # use numpy array for efficiency
fwd_array = np.empty(parYld_interp.shape)

for i, col in enumerate(parYld_interp.columns):
    par_rates = parYld_interp[col].values
    maturities = parYld_interp.index.values
    spot_array[:, i] = par_to_spot(par_rates, maturities)
    fwd_array[:, i] = spot_to_fwd(spot_array[:, i], maturities)

spot_df = pd.DataFrame(spot_array, index=parYld_interp.index, columns=parYld_interp.columns)
fwd_df = pd.DataFrame(fwd_array, index=parYld_interp.index, columns=parYld_interp.columns)